In [2]:
## data processsing
import json
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

############################# 
############ file path
filepath = "twitter.json"
filepath2 = "twits_raw_20180725.json"
#############################

def load_json_multiple(segments):
    chunk = ""
    news_list = []
    for segment in segments:
        chunk += segment
        try:
            yield json.loads(chunk)
            chunk = ""
        except ValueError:
            pass
def get_news_list(path):
    news_list = []
    with open(path) as f:
        for parsed_json in load_json_multiple(f):
            news_list.append(parsed_json)
    return news_list
def get_tag_list(twit_list):
    tag_list = []
    for twit in twit_list:
        for tag in twit['tags']:
            if tag not in tag_list and tag != 'NONE':
                tag_list.append(tag)
    return tag_list
def get_tagged_new_list(path):
    twit_list = get_news_list(path)
    tag_list = get_tag_list(twit_list)

    news_list_G9 = [ item for item in twit_list 
                    if bool(set(tag_list) & set(item['tags']))]

    news_list_none = [item for item in twit_list if item['tags'] == ['NONE']]
    return news_list_G9, news_list_none

tagged_news, none_tagged_news = get_tagged_new_list(filepath)
tagged_news2, none_tagged_news2 = get_tagged_new_list(filepath2)
newer_list = tagged_news2[-len(tagged_news2) + len(tagged_news):]
len(newer_list)

7568

In [3]:
sample_twits_list = newer_list.copy()
X = ["author: " + twit["author"] +" \n " + twit['text'] for twit in sample_twits_list]
y_raw = [twit["tags"] for twit in sample_twits_list]
mb = MultiLabelBinarizer()
y=mb.fit_transform(y_raw)
print(len(y_raw))
print(y_raw[0:10])
print(y[0:10])
def bin2dec(bin_list):
    int_num = 0
    for i, digit in enumerate(bin_list):
        int_num = int_num + digit*(2**i)
    return int_num

y_int = []
for item in y:
    y_int.append(bin2dec(item))
print(y_int[0:10])

7568
[['EUR'], ['USD'], ['EUR'], ['CNY'], ['USD', 'CNY'], ['GBP'], ['JPY'], ['CHF', 'USD'], ['EUR', 'USD'], ['USD', 'CNY']]
[[0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 1]
 [0 0 0 1 0 0 0 0 1]]
[16, 256, 16, 8, 264, 32, 64, 260, 272, 264]


In [ ]:
# num_train = int(0.8*len(X))
# X_train = X[:num_train]
# X_test = X[num_train:]
# # y_train = y[:num_train]
# # y_test = y[num_train:]
# y_train = y_int[:num_train]
# y_test = y_int[num_train:]

In [4]:
len(y_int)

7568

# load the packages

In [5]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import lightgbm as lgbm
import time
import pickle


# loading the model

In [15]:
import pickle
filename = 'multilabel_model_20180725.sav'
## test the whole  dataset 7.25
loaded_model = pickle.load(open(filename, 'rb'))
score = loaded_model.score(X, y_int)
print(score)

0.9867864693446089


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:

###  random forest

filename = 'model/multilabel_model_rf_20180725.sav'
## test the whole  dataset 
loaded_model_rf = pickle.load(open(filename, 'rb'))
score = loaded_model_rf.score(X, y_int)
print(score)

/Users/zhang/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


0.7704809725158562


In [5]:
### mlp
filename = 'model/multilabel_model_mlp_20180725.sav'
## test the whole  dataset 
loaded_model_rf = pickle.load(open(filename, 'rb'))
score = loaded_model_rf.score(X, y_int)
print(score)

0.676400634249


In [16]:
# vote 
filename = 'multilabel_model_vote_20180725.sav'
loaded_model_rf = pickle.load(open(filename, 'rb'))
score = loaded_model_rf.score(X, y_int)
print(score)

0.9892970401691332


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Error analysis

In [ ]:
## make the results back to binary
import numpy as np
y_pred = y_predicted.copy()
def dec2bin(num_list):
    bin_list = []
    for int_num in num_list:
        bin_sub_list = []
        for i in range(9):
    #         print(int_num)
            digit = int_num % 2
            int_num = int(int_num//2)
            bin_sub_list.append(digit)
        bin_list.append(bin_sub_list)
    return bin_list

data_ = {'twit': X_test, 
         'label': mb.inverse_transform(np.asarray(dec2bin(y_test))),
         'label_pred': mb.inverse_transform(np.asarray(dec2bin(y_pred))),
         'correct': y_pred == y_test}
df = pd.DataFrame(data=data_)
df.head()

# bin_num1 = [1,0,0,0,1,0,0,0,0]
# bin_num2 = [0,0,0,0,1,0,0,0,1]
# int_num = bin2dec(bin_num1)
# print(int_num)
# print(dec2bin([int_num]))
df.to_csv('data/results_20180725.csv', encoding='utf-8')

In [ ]:
df['twit'].iloc[0]